# Multinomial Distribution

---

## Import

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import statistics
import scipy
from scipy import stats

%matplotlib inline
plt.style.use("fivethirtyeight")

---

## Utilities

In [2]:
def get_discrete_ticks(xmin, xmax):
    cuts = [5, 10, 20, 50, 100, 200, 500, 1000]
    ticks = np.arange(xmin, xmax + 1)
    if len(ticks) > 15:
        for cut in cuts:
            ticks = [tick for tick in ticks if tick % cut == 0]
            if len(ticks) <= 15:
                return ticks
        return ticks
    return ticks

In [3]:
def get_bins(xmin, xmax, step = 1):
    bins = np.arange(xmin, xmax, step)
    bins = bins if len(bins) > 2 else get_bins(xmin, xmax + 1, step)
    return bins if len(bins) <= 100 \
        else get_bins(xmin, xmax, step = step + 1)

In [4]:
def plot_discrete_distribution(f, x_axis, obs = None, q = None, **kwargs):
    pmf = f.pmf(x_axis, **kwargs)
    fig, ax = plt.subplots(figsize = (8, 4))
    bars = ax.bar(x_axis, pmf, edgecolor = "k", linewidth = 2)
    
    print(f"Mean: {f.mean(**kwargs)}")
    print(f"Median: {f.median(**kwargs)}")
    print(f"Variance: {f.var(**kwargs)}")
    print(f"Standard Deviation: {f.std(**kwargs)}")
    print("-" * 10)
    if obs:
        plt.setp(bars[obs - x_axis.min()], color = "r", linewidth = 2)
        plt.setp(bars[obs - x_axis.min()], edgecolor = "k")
        obs_pmf = f.pmf(obs, **kwargs)
        print(f"PMF({obs}) = {obs_pmf}")
        
    plt.xticks(get_discrete_ticks(xmin = x_axis.min(), xmax = x_axis.max()))
    plt.show()

In [5]:
def sample_discrete_distribution(f, size = 1, seed = None, **kwargs):
    sample = f.rvs(size = size, random_state = seed, **kwargs)
    
    print(f"Min: {np.min(sample)}")
    print(f"Mean: {np.mean(sample)}")
    print(f"Median: {np.median(sample)}")
    print(f"Max: {np.max(sample)}")
    print(f"Variance: {np.var(sample, ddof = 0)}")
    print(f"Standard Deviation: {np.std(sample, ddof = 0)}")
    
    fig, ax = plt.subplots(figsize = (8, 4))
   
    plt.hist(x = sample,
             density = True,
             edgecolor = "k",
             bins = get_bins(sample.min(), sample.max() + 1),
             linewidth = 2)

    plt.xticks(get_discrete_ticks(sample.min(), sample.max()))
    plt.show()
    return sample

---

La distribuzione multinomiale è una generalizzazione della binomiale.

Nella binomiale ci sono solo due possibili *outcome* in ogni singolo trial, il successo e il fallimento. Nella multinomiale, è possibile avere più di due *outcome*.

Ad esempio, supponiamo che la distribuzione di gruppi sanguigni negli USA sia:
- 0: 0.44
- A: 0.42
- B: 0.10
- AB: 0.04

In un sample di 10 americani, qual è la probabilità di avere 6 "0", 2 "A", 1 "B" e 1 "AB"?

Posso definire la variabile aleatoria $X_i$ come il numero di occorrenze dell'*outcome* *i*. Di conseguenza ottengo:

$$\large P(X_1=x_1,\dots,X_k=x_k)=\frac{n!}{x_1!\dots x_k!}
p_1^{x_1}\cdot\cdot\cdotp_k^{x_k}$$

Dove chiaramente le variabili aleatorie i-esime possono assumere valori da 0 fino ad *n*, perché *n* sono i trial totali eseguiti.

Sappiamo anche che la somma di tutti i valori $x_i$ dev'essere pari ad *n*, perché gli *outcome* sono mutuamente esclusivi.

$$\large \sum_{i=1}^{k}x_i=n$$

E' come se ciascuna delle variabili aleatorie $X_i$ segua una propria distribuzione binomiale, caratterizzata dalla probabilità di occorrenza dell'*outcome* i-esimo e dal numero di trial totali.

$$\large E(X_i)=np_i\;\;\;Var(X_i)=np_i(1-p_i)$$

Tornando all'esempio, la probabilità che cerchiamo è:

$$P(X_1=6,X_2=2,X_3=1,X_4=1)=\frac{10!}{6!2!1!1!}0.44^6\cdot0.42^2
0.10^1\cdot0.04^1$$

In codice:

In [6]:
trials = 10
probabilities = [0.44, 0.42, 0.10, 0.04]
desired_outcomes = [6, 2, 1, 1]

stats.multinomial.pmf(x = desired_outcomes,
                      p = probabilities,
                      n = trials)

0.012902538743119897

---

**[Esempio]** Una sacca contiene 8 palline rosse, 3 gialle e 9 bianche. Si estraggono a caso 6 palline *with replacement*. Qual è la probabilità che 2 siano rossse, 1 gialla e 3 bianche?

In [7]:
trials = 6
probabilities = [8/20, 3/20, 9/20]
desired_outcomes = [2, 1, 3]

stats.multinomial.pmf(x = desired_outcomes,
                      p = probabilities,
                      n = trials)

0.1312200000000001

**Occhio:** Se il *sampling* fosse stato effettuato **without replacement**, avremmo dovuto usare la distribuzione **ipergeometrica multivariata**!

---